In [15]:
import pandas as pd
import numpy as np
from collections import Counter

In [16]:
prediction_files = [
        "output/kfold_output_0_2156.csv",
        "output/kfold_output_0_6468.csv",
]

In [17]:
predictions = []
for path in prediction_files:
    df = pd.read_csv(path)
    predictions.append(df)
    print(f"Loaded: {path}, Shape: {df.shape}")

Loaded: output/kfold_output_0_2156.csv, Shape: (869, 7)
Loaded: output/kfold_output_0_6468.csv, Shape: (869, 7)


In [18]:
len(predictions)

2

# soft voting (확률값 평균)

In [6]:
first_df = predictions[0]
result_df = pd.DataFrame()
result_df['id'] = first_df['id']
final_answers = []
final_probs = []

for idx, row_id in enumerate(result_df['id']):
    current_probs = []
    for pred_df in predictions:
        pred_row = pred_df[pred_df['id'] == row_id].iloc[0]
        probs = [pred_row[col] for col in pred_row.index if col.startswith('prob_')]
        probs = [0 if pd.isna(p) else p for p in probs]
        current_probs.append(probs)

    avg_probs = np.mean(current_probs, axis=0)
    answer = str(np.argmax(avg_probs) + 1)
        
    final_answers.append(answer)
    final_probs.append(avg_probs)

In [ ]:
result_df['answer'] = final_answers
result_df.to_csv("output/ensemble_result.csv")

In [8]:
result_with_prob = result_df.copy()
for i in range(len(final_probs[0])):
    result_with_prob[f'avg_prob_{i+1}'] = [probs[i] for probs in final_probs]

In [9]:
result_with_prob

,id,ensemble_answer,avg_prob_1,avg_prob_2,avg_prob_3,avg_prob_4,avg_prob_5
0,generation-for-nlp-0,1,0.922698,0.040491,0.014530,0.015717,6.563224e-03
1,generation-for-nlp-1,1,0.983166,0.007569,0.008182,0.001014,6.894798e-05
2,generation-for-nlp-2,4,0.000257,0.000252,0.000087,0.999399,3.236853e-06
3,generation-for-nlp-3,5,0.084100,0.054551,0.007359,0.013749,8.402416e-01
4,generation-for-nlp-4,3,0.237815,0.186903,0.417920,0.153744,3.618781e-03
...,...,...,...,...,...,...,...
864,generation-for-nlp-1609,1,0.999808,0.000151,0.000023,0.000014,3.536329e-06
865,generation-for-nlp-1512,1,0.999861,0.000128,0.000006,0.000004,6.436802e-07
866,generation-for-nlp-1382,3,0.000057,0.000081,0.999664,0.000193,5.344654e-06
867,generation-for-nlp-702,4,0.002915,0.000643,0.060629,0.935813,0.000000e+00


# Hard Voting (다수결)
- 동점이 있는 경우 동점이 있는 선택지 중 가장 확률값이 높은 것 선택

In [22]:
first_df = predictions[0]
result_df = pd.DataFrame()
result_df['id'] = first_df['id']

final_answers = []
for idx in range(len(first_df)):
    votes = [pred_df.iloc[idx]['answer'] for pred_df in predictions]
    vote_counts = Counter(votes)
    most_commons = vote_counts.most_common()
    
    #다수결로 공동 1등이 생겼으면
    if len(most_commons) > 1 and most_commons[0][1] == most_commons[1][1]:
        
        answer = [ans for ans, count in most_commons if count == most_commons[0][1]]
        probs = {}
        for ans in answer:
            probs[ans] = max(pred_df.iloc[idx][f'prob_{ans}'] for pred_df in predictions)
        selected_answer = max(probs.items(), key=lambda x: x[1])[0]
        final_answers.append(selected_answer)
    else:
        final_answers.append(most_commons[0][0])
        
result_df['answer'] = final_answers

In [23]:
vote_counts.keys()

dict_keys([1])

In [24]:
result_df.to_csv("output/ensemble_result.csv")